In [63]:
from scipy import stats
import pandas as pd
import numpy as np
import tqdm
import inspect
import warnings
warnings.filterwarnings('ignore')

def seleccionar_distribuciones(familia='realall', verbose=True):
    '''
    Esta función selecciona un subconjunto de las distribuciones disponibles
    en scipy.stats
    
    Parameters
    ----------
    familia : {'realall', 'realline', 'realplus', 'real0to1', 'discreta'}
        realall: distribuciones de la familia `realline` + `realplus`
        realline: distribuciones continuas en el dominio (-inf, +inf)
        realplus: distribuciones continuas en el dominio [0, +inf)
        real0to1: distribuciones continuas en el dominio [0,1]
        discreta: distribuciones discretas
        
    verbose : bool
        Si se muestra información de las distribuciones seleccionadas
        (the default `True`).
        
    Returns
    -------
    distribuciones: list
        listado con las distribuciones (los objetos) seleccionados.
        
    Raises
    ------
    Exception
        Si `familia` es distinto de 'realall', 'realline', 'realplus', 'real0to1',
        o 'discreta'.
        
    Notes
    -----
        Las distribuciones levy_stable y vonmises han sido excluidas por el momento.

    '''
    
    distribuciones = [getattr(stats,d) for d in dir(stats) \
                     if isinstance(getattr(stats,d), (stats.rv_continuous, stats.rv_discrete))]
    
    exclusiones = ['levy_stable', 'vonmises']
    distribuciones = [dist for dist in distribuciones if dist.name not in exclusiones]
            
    dominios = {
        'realall' : [-np.inf, np.inf],
        'realline': [np.inf,np.inf],
        'realplus': [0, np.inf],
        'real0to1': [0, 1], 
        'discreta': [None, None],
    }

    distribucion = []
    tipo = []
    dominio_inf = []
    dominio_sup = []

    for dist in distribuciones:
        distribucion.append(dist.name)
        tipo.append(np.where(isinstance(dist, stats.rv_continuous), 'continua', 'discreta'))
        dominio_inf.append(dist.a)
        dominio_sup.append(dist.b)
    
    info_distribuciones = pd.DataFrame({
                            'distribucion': distribucion,
                            'tipo': tipo,
                            'dominio_inf': dominio_inf,
                            'dominio_sup': dominio_sup
                          })

    info_distribuciones = info_distribuciones \
                          .sort_values(by=['dominio_inf', 'dominio_sup'])\
                          .reset_index(drop=True)
    
    if familia in ['realall', 'realline', 'realplus', 'real0to1']:
        info_distribuciones = info_distribuciones[info_distribuciones['tipo']=='continua']
        condicion = (info_distribuciones['dominio_inf'] == dominios[familia][0]) & \
                    (info_distribuciones['dominio_sup'] == dominios[familia][1]) 
        info_distribuciones = info_distribuciones[condicion].reset_index(drop=True)
        
    if familia in ['discreta']:
        info_distribuciones = info_distribuciones[info_distribuciones['tipo']=='discreta']
        
    seleccion = [dist for dist in distribuciones \
                 if dist.name in info_distribuciones['distribucion'].values]
    
    
    if verbose:
        print("---------------------------------------------------")
        print("       Distribuciones seleccionadas                ")
        print("---------------------------------------------------")
        with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
            print(info_distribuciones)
    
    return seleccion


def comparar_distribuciones(x, familia='realall', ordenar='aic', verbose=True):
    '''
    Esta función selecciona y ajusta un subconjunto de las distribuciones 
    disponibles en scipy.stats. Para cada distribución calcula los valores de
    Log Likelihood, AIC y BIC.
    
    Parameters
    ----------
    x : array_like
        datos con los que ajustar la distribución.
        
    familia : {'realall', 'realline', 'realplus', 'real0to1', 'discreta'}
        realall: distribuciones de la familia `realline` + `realplus`
        realline: distribuciones continuas en el dominio (-inf, +inf)
        realplus: distribuciones continuas en el dominio [0, +inf)
        real0to1: distribuciones continuas en el dominio [0,1]
        discreta: distribuciones discretas
    
    ordenar : {'aic', 'bic'}
        criterio de ordenación de mejor a peor ajuste.
    
    verbose : bool
        Si se muestra información de las distribuciones seleccionadas
        (the default `True`).
        
    Returns
    -------
    resultados: data.frame
        distribucion: nombre de la distribución.
        log_likelihood: logaritmo del likelihood del ajuste.
        aic: métrica AIC.
        bic: métrica BIC.
        n_parametros: número de parámetros de la distribución de la distribución.
        parametros: parámetros del tras el ajuste
        
    Raises
    ------
    Exception
        Si `familia` es distinto de 'realall', 'realline', 'realplus', 'real0to1',
        o 'discreta'.
        
    Notes
    -----

    '''
    
    distribuciones = seleccionar_distribuciones(familia=familia, verbose=verbose)
    distribucion_ = []
    log_likelihood_= []
    aic_ = []
    bic_ = []
    n_parametros_ = []
    parametros_ = []
    
    for i, distribucion in enumerate(distribuciones):
        
        print(f"{i+1}/{len(distribuciones)} Ajustando distribución: {distribucion.name}")
        
        try:
            parametros = distribucion.fit(data=x)
            nombre_parametros = [p for p in inspect.signature(distribucion._pdf).parameters if not p=='x'] + ["loc","scale"]
            parametros_dict = dict(zip(nombre_parametros, parametros))
            log_likelihood = distribucion.logpdf(x, *parametros).sum()
            aic = -2 * log_likelihood + 2 * len(parametros)
            bic = -2 * log_likelihood + np.log(x.shape[0]) * len(parametros)
            
            distribucion_.append(distribucion.name)
            log_likelihood_.append(log_likelihood)
            aic_.append(aic)
            bic_.append(bic)
            n_parametros_.append(len(parametros))
            parametros_.append(parametros_dict)
            
            resultados = pd.DataFrame({
                            'distribucion': distribucion_,
                            'log_likelihood': log_likelihood_,
                            'aic': aic_,
                            'bic': bic_,
                            'n_parametros': n_parametros_,
                            'parametros': parametros_,
                
                         })
            
            resultados = resultados.sort_values(by=ordenar).reset_index(drop=True)
            
        except Exception as e:
            print(f"Error al tratar de ajustar la distribución {distribucion.name}")
            print(e)
            print("")
            
    return resultados

In [66]:
import numpy as np


def nvidia(k):
    return np.random.normal(0, 3, size=k)


def yahoo(k):
    return np.random.normal(0, 1,size = k)




years = 10 
days = 365 * years

x_0 = 1000

sacrifice_amount = 200  # 20 por ciento
sacrifice_days = 100

nvidia_sample = nvidia(sacrifice_days)
yahoo_sample = yahoo(sacrifice_days)


nvidia_dist = comparar_distribuciones(nvidia_sample)
yahoo_dist = comparar_distribuciones(yahoo_sample)

---------------------------------------------------
       Distribuciones seleccionadas                
---------------------------------------------------
          distribucion      tipo  dominio_inf  dominio_sup
0               cauchy  continua         -inf          inf
1          crystalball  continua         -inf          inf
2               dgamma  continua         -inf          inf
3             dweibull  continua         -inf          inf
4            exponnorm  continua         -inf          inf
5           genextreme  continua         -inf          inf
6        genhyperbolic  continua         -inf          inf
7          genlogistic  continua         -inf          inf
8              gennorm  continua         -inf          inf
9             gumbel_l  continua         -inf          inf
10            gumbel_r  continua         -inf          inf
11           hypsecant  continua         -inf          inf
12           jf_skew_t  continua         -inf          inf
13           johns

In [71]:
yahoo_dist_top = yahoo_dist.loc[[0,1,2]]
nvidia_dist_top = nvidia_dist.loc[[0,1,2]]

yahoo_dist_top, nvidia_dist_top

(  distribucion  log_likelihood         aic         bic  n_parametros  \
 0     dweibull     -138.966196  283.932392  291.747902             3   
 1         norm     -140.072343  284.144686  289.355026             2   
 2       dgamma     -139.210111  284.420222  292.235733             3   
 
                                           parametros  
 0  {'c': 1.3471332224618193, 'loc': 0.06379410301...  
 1  {'loc': 0.027237624389043754, 'scale': 0.98194...  
 2  {'a': 1.614183787646474, 'loc': 0.064129535684...  ,
   distribucion  log_likelihood         aic         bic  n_parametros  \
 0         norm     -245.958982  495.917964  501.128304             2   
 1   genextreme     -245.041308  496.082615  503.898126             3   
 2     skewnorm     -245.280702  496.561404  504.376914             3   
 
                                           parametros  
 0  {'loc': 0.1347200376406246, 'scale': 2.8310602...  
 1  {'c': 0.22418246464784708, 'loc': -0.932759274...  
 2  {'a': 1.5854082

In [126]:
nvdist = nvidia_dist_top.loc[0]
yhdist = yahoo_dist_top.loc[1]

nvdist

x = stats.norm()
y = stats.norm()
nvdist_pars = nvdist['parametros']
yhdist_pars = nvdist['parametros']

def nv_generator():
    return nvdist_pars['loc'] + x.rvs() * nvdist_pars['scale']

def yh_generator():
    return yhdist_pars['loc'] + y.rvs() * yhdist_pars['scale']


def sacrifice(amount, actions):
    n_acts = len(actions)
    act_0 = amount // n_acts
    rest = amount % n_acts
    money = act_0 * np.ones(n_acts)
    k = 0
    print(money)
    history = []
    while money.sum() > 0:
        record = [nvidia_sample[k], yahoo_sample[k]]
        money -= np.abs(record)
        history.append(record)
        for k in range(len(money)):
            if money[k] < 0:
                money[k] = 0
        k = k + 1 
        print(money)
    return history, money


sacrifice(211, ['A','B'])




[105. 105.]
[101.51884826 103.25426173]
[100.9061039  102.56958837]
[100.29335954 101.884915  ]
[ 99.68061518 101.20024164]
[ 99.06787082 100.51556828]
[98.45512646 99.83089492]
[97.8423821  99.14622155]
[97.22963774 98.46154819]
[96.61689338 97.77687483]
[96.00414902 97.09220146]
[95.39140466 96.4075281 ]
[94.7786603  95.72285474]
[94.16591594 95.03818138]
[93.55317158 94.35350801]
[92.94042722 93.66883465]
[92.32768286 92.98416129]
[91.7149385  92.29948792]
[91.10219414 91.61481456]
[90.48944978 90.9301412 ]
[89.87670542 90.24546784]
[89.26396106 89.56079447]
[88.6512167  88.87612111]
[88.03847234 88.19144775]
[87.42572798 87.50677439]
[86.81298362 86.82210102]
[86.20023926 86.13742766]
[85.5874949 85.4527543]
[84.97475054 84.76808093]
[84.36200618 84.08340757]
[83.74926182 83.39873421]
[83.13651746 82.71406085]
[82.5237731  82.02938748]
[81.91102874 81.34471412]
[81.29828438 80.66004076]
[80.68554002 79.97536739]
[80.07279566 79.29069403]
[79.4600513  78.60602067]
[78.84730694 77.92

([[3.4811517418814737, 1.7457382697490629],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.6846733628289123],
  [-0.6127443600272962, -0.684673362